<a href="https://colab.research.google.com/github/mvmorr/CE888/blob/master/lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv",header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:

df = df.drop(df.columns[0],axis=1)
arr = df.values
new_arr, idx = replace(df.values, 0.1)


In [5]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 2
ratings = new_arr
latent_preferences = np.random.random((ratings.shape[0],n_latent_factors))
latent_features = np.random.random((ratings.shape[1], n_latent_factors))

In [7]:
latent_features

array([[0.55926211, 0.38612939],
       [0.52468851, 0.56231425],
       [0.25275699, 0.89327027],
       [0.58551292, 0.16654205],
       [0.87057143, 0.94875207],
       [0.74865226, 0.41248467],
       [0.23163929, 0.93442526],
       [0.37036166, 0.95263094],
       [0.25846485, 0.87677384],
       [0.57085749, 0.12985598],
       [0.23826325, 0.23841608],
       [0.86955577, 0.57986326],
       [0.87361063, 0.03711057],
       [0.76109208, 0.03058966],
       [0.38372967, 0.07968203],
       [0.31177941, 0.75254641],
       [0.88858918, 0.58649317],
       [0.15966534, 0.9239489 ],
       [0.47635668, 0.34012356],
       [0.73399305, 0.70816189],
       [0.00439988, 0.80867228],
       [0.19354332, 0.16244474],
       [0.33154348, 0.77620658],
       [0.28530999, 0.2062624 ],
       [0.29285377, 0.15202009],
       [0.10907995, 0.45522173],
       [0.01598056, 0.68801976],
       [0.67561127, 0.02835802],
       [0.89806783, 0.06214197],
       [0.74131206, 0.89813356],
       [0.

In [8]:
latent_preferences

array([[0.29791772, 0.72142976],
       [0.66616916, 0.24397559],
       [0.23820616, 0.73626377],
       ...,
       [0.15765339, 0.22511866],
       [0.95387201, 0.2181586 ],
       [0.46283959, 0.14008091]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_preferences[user_id]
    item_preference = latent_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_preferences[user_id][:]
    latent_preferences[user_id] -= alpha * err * latent_features[item_id]
    latent_features[item_id] -= alpha * err * user_pref_values
    return err

def sgd(iterations=25):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_preferences.shape[0]):
            for item_id in range(latent_features.shape[0]):
                rating = ratings[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

In [10]:
sgd()

26.285173254083638
25.302865352506966
25.01120233057793
24.792820670573427
24.581657326703482
24.368936323893088
24.153853724697843
23.937216543554538
23.72027510913509
23.504409036709465
23.29099998144739
23.081356689162924
22.876659801433817
22.677917806889038
22.485931557020745
22.3012659240596
22.124226944112856
21.95484226454387
21.792842318069763
21.63763958847126
21.488303797808424
21.34353208728177
21.201615680044103
21.06040867698749
20.917311227643445


In [11]:
predictions = latent_preferences.dot(latent_features.T)
predictions

array([[0.99247476, 0.9301195 , 0.88063911, ..., 1.23627784, 0.80491885,
        1.41956314],
       [1.20560455, 1.06338225, 0.98327958, ..., 1.3819475 , 0.85068244,
        1.77741489],
       [3.90108204, 3.66655127, 3.47523915, ..., 4.87843445, 3.18406456,
        5.57139616],
       ...,
       [0.25994209, 0.24355878, 0.23058369, ..., 0.32370421, 0.21072001,
        0.37184329],
       [1.00202532, 0.80019036, 0.70845789, ..., 0.99786082, 0.54715535,
        1.54396235],
       [2.26383364, 2.04145543, 1.9044823 , ..., 2.67549486, 1.6827964 ,
        3.30192779]])

In [0]:
values = [zip(ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [0]:
comparison_data

,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,-8.5.1,-6.75,-7.18,8.45,-7.18.1,-7.52.1,-7.43,-9.81,-9.85.1,-9.85.2,-9.37,1.5,-4.37,-9.81.1,-8.5.2,1.12,7.82,2.86,9.13,-7.43.1,2.14,-4.08,-9.08,7.82.1,5.05,4.95,-9.17,-8.4,-8.4.1,-8.4.2,...,8.59,3.59,-6.84,-9.03.1,2.82.1,-1.36,-9.08.2,8.3.1,5.68,-4.81,99,99.1,99.2,99.3,99.4,99.5,99.6,-9.42.1,99.7,99.8,99.9,-7.72,99.10,99.11,99.12,99.13,99.14,99.15,99.16,99.17,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,"(99.0, 1.4136581523041103)","(-0.29, 1.2527165304742207)","(99.0, 1.1631921812820512)","(4.37, 0.11173119835435433)","(99.0, 1.0844498044956334)","(-9.66, 2.3344751173864364)","(-0.73, 0.4131091900410161)","(-5.34, -0.0468136040836036)","(8.88, 0.49967492887340853)","(9.22, 1.6310450357342614)","(6.75, 2.3660197796495406)","(8.64, 2.3830759435008044)","(4.42, -0.7484698079289159)","(7.43, 2.3949733468623626)","(4.56, -0.9324622087158546)","(-0.97, -1.849418028951186)","(4.66, -0.13068473874603032)","(-0.68, 0.07589781249225967)","(3.3, 0.9547669601178774)","(-1.21, 0.25737533410641644)","(0.87, 2.793581326429572)","(8.64, 1.769702148862473)","(8.35, 1.1149592257239418)","(9.17, -0.30170267792414035)","(0.05, 1.5080834224611885)","(99.0, 2.331597602222864)","(4.71, 3.2791746763382017)","(0.87, 2.268782392764046)","(-0.39, 3.112538184843688)","(6.99, 0.9104398836654619)","(6.5, 2.80207318138724)","(99.0, 3.2705536799948494)","(7.14, 0.12965445534625664)","(9.03, 1.9086841958054543)","(-1.8, 3.2152408778426924)","(0.73, 3.237302112099262)","(7.09, -0.1283104371278887)","(99.0, 1.8963042584812195)","(-0.87, 2.1040322176416213)","(7.91, 1.9143260987478579)",...,"(99.0, 2.9929107297072868)","(-3.35, 3.184799631368682)","(-9.03, 1.3352527962949765)","(4.47, 0.36588861661288585)","(4.08, 2.8480397941885616)","(-3.83, 2.9560174147479708)","(8.74, 0.3268192234105916)","(1.12, 2.9437790018276893)","(99.0, 3.0170408210941186)","(7.52, 1.530222546044464)","(-5.0, 0.6435212914942392)","(2.77, 2.971493145990525)","(8.3, 2.0289621409274714)","(7.77, 0.1023213476471515)","(7.33, 0.9120243500553294)","(6.21, 2.698105384104206)","(7.72, 1.6440858994829617)","(8.98, 2.3656992467288105)","(8.64, 1.3744693553974643)","(8.2, 2.148322885710139)","(3.93, 2.3450201304605884)","(4.85, 1.6870080960276568)","(4.85, 2.5572536672840593)","(6.07, 1.6107865693226453)","(8.98, 1.8867228008032046)","(4.51, 0.9968134256634315)","(99.0, 2.289828737429393)","(3.69, 2.370484839817463)","(4.56, 3.292431231977069)","(0.58, 1.5527597743508843)","(99.0, 2.258631582314343)","(-4.95, 1.868512312227433)","(-0.29, 2.7127415549801)","(7.86, 1.6252863766706207)","(-0.19, 1.591234682999782)","(-2.14, 2.0174129881953116)","(3.06, 2.041288513794386)","(0.34, 1.7520624812651509)","(99.0, 1.0333272101007127)","(1.07, 2.0942129851614593)"
1,"(99.0, 3.547008068469113)","(99.0, 3.4091034024088414)","(99.0, 3.3277118582616083)","(99.0, 1.380521275903164)","(9.03, 2.780724513468996)","(9.27, 5.521222826391805)","(9.03, 1.4810652941915083)","(9.27, 0.35471163055596155)","(99.0, 2.1312059605186326)","(99.0, 4.120076717444776)","(99.0, 5.507119327118539)","(7.57, 5.673831067296458)","(9.37, -0.9294807328270801)","(6.17, 5.625869117841096)","(-6.36, -1.2484635868110974)","(-6.89, -3.0016311854424504)","(-7.86, 0.2884178175762434)","(9.03, 0.7246074451657794)","(99.0, 2.583673407210185)","(9.03, 1.340372102259995)","(99.0, 6.312937847747392)","(99.0, 4.485138211898036)","(8.25, 3.2746230306201567)","(99.0, 0.642575021553124)","(99.0, 4.021758715434996)","(7.48, 5.434294298336981)","(99.0, 6.985716743588113)","(99.0, 5.3186322671610196)","(8.93, 6.703164907792167)","(99.0, 3.005360290962481)","(6.17, 6.195884284917441)","(7.28, 6.932765650103476)","(99.0, 1.4609752517382202)","(99.0, 4.800688784109634)","(8.98, 6.942327596890806)","(7.33, 6.813615213066803)","(99.0, 0.8063147587162455)","(6.17, 4.633171705058302)","(9.08, 5.1261498209919685)","(7.33, 4.737704690419934)",...,"(99.0, 6.654560889244653)","(7.28, 6.9023

In [12]:
x, y = idx
len(x)

181045

In [13]:
error = 0
for n in range(len(idx[0])):
  error+=(df.values[x[n]][y[n]]- predictions[x[n]][y[n]])**2
  if n % 100 == 0:
    print(error)


74.87694461117839
2402.5369975113667
4469.187305952994
6760.100134608052
8758.147469711459
10605.444540371765
12848.006264272715
14637.637207245287
16721.73229393905
18917.239285782623
21356.831917763207
23573.740917716623
25895.490085144822
28131.237739722423
30130.266409533186
32269.898656380534
33964.52941671693
36447.20491722332
38346.90953256694
40712.165806859615
42398.282697172486
44104.46435165021
46157.63532041777
47967.15174373899
49983.906107344796
51971.07564460409
54445.52047996879
56558.37344031321
58582.11856108815
61146.13653299954
63436.46239292668
65656.40410137773
67539.9877862958
69880.77072300532
71965.06158678365
73812.6684132345
76024.94253923233
78322.40789509538
80498.43102201023
82639.93670442574
84837.51160376854
86905.58239943872
89127.23124133407
91697.92261369817
94195.98185879008
96409.99962373731
98521.91615882868
100946.10020531803
103330.17543448263
105334.88632167372
106895.9160685651
108961.94235965222
111465.95848358108
114007.85734644577
116538.039

In [14]:
error/len(x)

21.207876631119493